### Load words with concreteness score from xls file, train a classifier to classify words as concrete or abstract, save the classifier

In [33]:
import numpy as np
import spacy
from nltk.corpus import wordnet as wn

import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# spacy.cli.download("en_core_web_md")
nlp = spacy.load("en_core_web_md")

In [13]:
# TODO tune concreteness threshold
concreteness_df = pd.read_excel("data/brysbaert_concreteness_ratings.xlsx", na_filter=False)
# extract list of words with concreteness >= 3 and < 3
concrete_words = concreteness_df.query('conc_score >= 3')['word'].tolist()  # words with concreteness >= 3
abstract_words = concreteness_df.query('conc_score < 3')['word'].tolist()  # words with concreteness < 3

In [14]:
print(len(concrete_words))
print(concrete_words[:10])
print()
print(len(abstract_words))
print(abstract_words[:10])

18776
['accumulate', 'add', 'aerially', 'ahead', 'aiming', 'airless', 'alternation', 'anaphylactic', 'anatomically', 'annotate']

21178
['eh', 'essentialness', 'although', 'spirituality', 'would', 'spiritually', 'whatsoever', 'conceptualistic', 'conventionalism', 'belief']


In [15]:
classes = ['concrete', 'abstract']
train_set = []
train_set.append(concrete_words)
train_set.append(abstract_words)

In [29]:
from time import sleep

# get word vectors list
X = np.array([])
index = 0
for part in train_set:
    for word in part:
        X.append(nlp(word)[0].vector)

In [11]:
# get labels
y = [label for label, part in enumerate(train_set) for _ in part]
classifier = LogisticRegression(C=0.1, class_weight='balanced').fit(X, y)

NameError: name 'train_set' is not defined

In [18]:
# save classifier
joblib.dump(classifier, "trained_models/concrete_abstract_classifier.joblib")

['trained_models/concrete_abstract_classifier.joblib']

In [19]:
# load classifier
classifier = joblib.load("trained_models/concrete_abstract_classifier.joblib")

In [20]:
synsets = ['war.n.01', 'fiefdom.n.01', 'bed.n.03', 'return_on_invested_capital.n.01', 'texture.n.02', 'news.n.01',
           'look.n.02']

for synset_str in synsets:
    synset = wn.synset(synset_str)
    synset_name = synset.lemma_names()[0]
    synset_vector = list(nlp(synset_name))[0].vector
    synset_class = classifier.predict([synset_vector])[0]
    # print classification
    print(f'{synset_name} -> {synset_class} - {classes[synset_class]}')

war -> 1 - abstract
fiefdom -> 1 - abstract
bed -> 0 - concrete
return_on_invested_capital -> 1 - abstract
texture -> 1 - abstract
news -> 0 - concrete
look -> 0 - concrete


### Regression version. Predicting concreteness score of a word.

In [30]:
new_y = concreteness_df['conc_score']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.3, random_state=42)

model = LinearRegression().fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.9317894584297259


In [24]:
#print training set
print(len(X))
print(len(X_train))

39954
31963


In [ ]:
from sklearn import linear_model

alphas = np.logspace(-5, 1, 60)
enet = linear_model.ElasticNet(l1_ratio=0.7, max_iter=10000)
train_errors = list()
test_errors = list()
for alpha in alphas:
    enet.set_params(alpha=alpha)
    enet.fit(X_train, y_train)
    train_errors.append(enet.score(X_train, y_train))
    test_errors.append(enet.score(X_test, y_test))

i_alpha_optim = np.argmax(test_errors)
alpha_optim = alphas[i_alpha_optim]
print("Optimal regularization parameter : %s" % alpha_optim)

# Estimate the coef_ on full data with optimal regularization parameter
enet.set_params(alpha=alpha_optim)
coef_ = enet.fit(X, new_y).coef_

C:\Users\Gianl\Desktop\UniTo\tln\dicaro\basicness_classification\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.763e+00, tolerance: 3.008e+00
  model = cd_fast.enet_coordinate_descent(
